# Imports

In [6]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from skimage import io
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from imageProcessing import mp_detection, formatPoints

In [ ]:
#### create array to signify the actions for iterating

# actions to be detected by model
actions = np.array(['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

# Preprocess Data and create labels

In [ ]:
#### create all the path variables

# path for exported data (numpy arrays)
DATA_PATH = os.path.join('MP_DATA')

# the path for the data used to train our model
TRAIN_PATH = os.path.join(DATA_PATH, 'asl_alphabet_train')

# the path for the data used to test our model
TEST_PATH = os.path.join(DATA_PATH, 'asl_alphabet_test')

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []

# the number of files we are loading in from our converted numpy dataset to train out model
numTrainFiles = 1000
for action in actions:
    count = 0 
    for filename in os.listdir(os.path.join(TRAIN_PATH, action)):
        window = []
        keypoints = np.load(os.path.join(TRAIN_PATH, action, filename))
        window.append(keypoints)
        sequences.append(window)
        labels.append(label_map[action])
        count += 1
        if numTrainFiles == count:
            print(f"Loading Sequences and labels: '{action}' complete")
            break

In [ ]:
# the number of files we are choosing to train the model at the moment
# this should match our length of sequences
numTrainFiles*(len(actions))

In [ ]:
print(f"This should be {len(sequences)} if numTrainFiles is {numTrainFiles}: {len(y)}")
print(f"This should be 29 always: {len(y[0])}")

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Splitting our data into training and testing partitions for our model training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

# Build and Train LSTM Neural Network

In [ ]:
# setting up the console logs to track info while training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# creating and building the model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1,126)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Here we train the model
model.fit(X_train, y_train,epochs=2000, callbacks=[tb_callback])

In [ ]:
# Here we save the model to be used later 
model.save("asl_Alphabet_Model_v.0.1.h5")

# Evaluation using confusion matrix and accuracy

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
# matrix representation for checking model accuracy
multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
# percentage accuracy
accuracy_score(ytrue,yhat)